### Get TikTok data

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
from datetime import timedelta

In [2]:
today = (dt.datetime.today() + timedelta(days=30)).strftime('%Y-%m-%d')
today

'2022-02-10'

In [3]:
payload={}
headers = {
  'X-API-KEY': 'QT8K1SQ333HurjoXkTVKOchKswsI83cS'
}

In [4]:
count = '30'
secUid = 'MS4wLjABAAAAn982fxeWVi0MRU_Q0Lg0mVJ9AmAZ-GvWgjy9rSGReFPs_xkQSFkKoUnkuWTVhryi'

In [5]:
rng = pd.date_range(start='2021-03-01', end = today, freq='M')
df = pd.DataFrame({ 'date' : rng }, )

In [6]:
df['cursor'] = (pd.to_datetime(df['date']) - dt.datetime(1970,1,1)).dt.total_seconds().round().astype(str).str.replace('.0', '', regex=False) + '000'

In [7]:
cursors = df['cursor'].to_list()

In [8]:
datas = []

for c in cursors:
    url = f"https://api.tikapi.io/public/posts?count={count}&secUid={secUid}&cursor={c}"
    r = requests.request("GET", url, headers=headers, data=payload)
    datas.append(r.json())

In [9]:
len(datas)

11

In [10]:
def simple_dict(tiktok_dict):
    to_return = {}
    to_return['user_name'] = d['author']['uniqueId']
    to_return['video_id'] = d['id']
    to_return['video_desc'] = d['desc']
    to_return['video_time'] = d['createTime']
    to_return['video_duration'] = d['video']['duration']
    to_return['n_shares'] = d['stats']['shareCount']
    to_return['n_likes'] = d['stats']['diggCount']
    to_return['n_comments'] = d['stats']['commentCount']
    to_return['n_plays'] = d['stats']['playCount']
    to_return['video_link'] = 'https://www.tiktok.com/@{}/video/{}?lang=en'.format(to_return['user_name'], to_return['video_id'])
    return to_return

In [11]:
dataframe = []

for data in datas:
    for d in data['itemList']:
        dataframe.append(simple_dict(d))

In [12]:
df = pd.DataFrame(dataframe)

In [13]:
df = df.sort_values('video_time', ascending=False).drop_duplicates(subset='video_id').reset_index(drop=True)

In [17]:
df.head()

,user_name,video_id,video_desc,video_time,video_duration,n_shares,n_likes,n_comments,n_plays,video_link
0,jongraz,7051972968347503919,plz plan/adjust your day accordingly #bones #n...,1641915407,58,2000,47500,1671,168900,https://www.tiktok.com/@jongraz/video/70519729...
1,jongraz,7051592308818447662,Reply to @szabochick ☹️,1641826778,59,266,92800,7291,605500,https://www.tiktok.com/@jongraz/video/70515923...
2,jongraz,7051388623291338031,handsome ✅ stylish ✅ dry ✅,1641779352,32,3147,132000,1754,632800,https://www.tiktok.com/@jongraz/video/70513886...
3,jongraz,7050491114775842095,we must HEED THE BONES!! 🔮🦴🔮,1641570386,45,4190,97900,1700,437500,https://www.tiktok.com/@jongraz/video/70504911...
4,jongraz,7049430483209391366,let’s help some SWEET BABIES* *animals!! #anim...,1641323438,57,653,76100,9938,442400,https://www.tiktok.com/@jongraz/video/70494304...


In [15]:
len(df)

160

In [16]:
df.to_csv('data/processed/jongraz_videos.csv',index=False)